## 1. Importing Libraries

In [ ]:
import warnings; warnings.simplefilter('ignore') 
import pandas as pd
import numpy as np
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time

import math
import matplotlib.dates as mdates
from datetime import time, date, timedelta
import datetime as dt

In [ ]:
## Defining directories 
## Place change the directory as requred
ipath= "EE 526 Project Dataset and Results/"

## 2. Querring Wejo connected vehicle data from "Intrans" athena

In [ ]:
## Selected crash locations, 
crs=pd.read_csv(path+"Crash Locations.csv")

In [ ]:
## Definig SQL query for exact measure and generating required fields

query = """
select datapointid, journeyid, capturedtimestamp, latitude, longitude,  speed, acceleration, heading, route_id,  dist+segment_start_measure as measure,  year, month, day, hour 
from (
    SELECT * , round((ST_Distance(ST_POINT(t.longitude,t.latitude), ST_Point(t.segment_start_longitude,t.segment_start_latitude))*69.047),5) dist  
    from (
        select datapointid, journeyid, capturedtimestamp, latitude, longitude, geohash, postalcode, speed, acceleration, heading, vm.route_id, distance_from_route, vm.segment_start_measure, vm.segment_end_measure, segment_start_longitude, segment_start_latitude, year, quarter, vm.county, month, day, hour 
        from (
            select * from "wejo_vm"."2022_q4" where county = %(cnt)s and month = %(mo)s and day= %(dy)s and (hour between %(hr)s-2 and %(hr)s +2) and route_id = %(rid)s and (segment_start_measure between %(sm)s - 2 and %(sm)s + 2) and (distance_from_route between -50 and 50 )) vm
            left join "lrs_geohash"."ft_cnt_gh_bearing" lrs 
                    on lrs.geohash_lrs = vm.geohash and lrs.route_id = vm.route_id and lrs.county = vm.county and 
                    (lrs.segment_start_measure between vm.segment_start_measure-0.002 and vm.segment_start_measure+0.002) 
                    and (lrs.segment_end_measure between vm.segment_end_measure-0.002 and vm.segment_end_measure+0.002)
            ) t
        )
            """

In [ ]:
## Extracting wejo connected vehicle data in a loop for every crash at the defined boundaries
st = time.time()
cry=crs[(crs["year"]==2022)]
for m in range(10,13):
    crm=cry[(cry["month"]==m)].reset_index(inplace=False)
    for i in range(0,len(crm)):
        cnt = crm.at[i,"NAME"].upper()
        dy = crm.at[i,"day"]
        rid = crm.at[i,"ROUTE_ID"]
        ck = crm.at[i,"atmid"]
        hr = crm.at[i,"hour"]
        sm = crm.at[i,"seg_st_mea"]
        dicn = {'mo' : m,'cnt': str(cnt), 'dy': int(dy), 'rid':str(rid), 'sm':float(sm), 'hr': int(hr)}
        wejo_df=as_pandas(cursor.execute(query,dicn))
        
        wejo_df.to_csv(path+'Data/Crash Locations/'+str(ck)+'-wejo.csv',index=False) # The storage location is not provided as these csv files are not submitted  
        
        print('Execution time for month',m, "Key", ck, ":", time.time() - st, 'seconds')

### 4 hour images

In [ ]:
import glob
files = glob.glob(ipath + "Evaluation/Data/*.csv") # this location is also not provided as these files are not submitted, final images are submitted 
li = []
for filename in files:
    fn=filename[-19:-9]
    try:
        df = pd.read_csv(filename, index_col=None, header=0)
        df['capturedtimestamp'] = pd.to_datetime(df['capturedtimestamp'])
        df1=df[[ 'journeyid', 'capturedtimestamp', 'measure']]
        df1.set_index('capturedtimestamp', inplace=True)

        fig, ax = plt.subplots(figsize=(150, 50))
        df1.groupby('journeyid')['measure'].plot(linewidth=4)
        ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[0,15,30,45], interval = 1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.rc('xtick',labelsize=50)
        plt.rc('ytick',labelsize=50)
        plt.grid()
        plt.savefig(ipath+"Evaluation/Images/"+str(fn)+"-Trajectory.jpg",bbox_inches='tight', dpi=100)

        print("completed: ",fn) #, end="\r"
    except:
        li.append(fn)
        print("Failed: ",fn) # , end="\r"

## 3. Geneerating overall crash scenario Plots for all locations

In [ ]:
# Importing crash data
crs=pd.read_csv(path+"Crash Locations.csv")
crs['datetime'] = pd.to_datetime(crs['datetime'])

In [ ]:
## Calling each file saved from the previous section and plotting the trajectories and saving them in a driectory
failed = []
st = time.time()
cry=crs[(crs["year"]==2022)]
for m in range(10,11):
    crm=cry[(cry["month"]==m)].reset_index(inplace=False)
    for i in range(0,len(crm)):
        ck = crm.at[i,"atmid"]
        ct = crm.at[i,"datetime"]
        try:
            df = pd.read_csv(path+'Data/Crash Locations/'+str(ck)+'-wejo.csv')
            df['capturedtimestamp'] = pd.to_datetime(df['capturedtimestamp'])
            df1=df[[ 'journeyid', 'capturedtimestamp', 'measure']]
            df1.set_index('capturedtimestamp', inplace=True)
        
            fig, ax = plt.subplots(figsize=(150, 50))
            df1.groupby('journeyid')['measure'].plot(linewidth=4)
            ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[0,15,30,45], interval = 1))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
            plt.axvline(ct,color='b', linewidth=10,linestyle='--',ymax =50, label='ATMS_Crash_Time: {}'.format(ct))
            plt.rc('xtick',labelsize=50)
            plt.rc('ytick',labelsize=50)
            plt.grid()
            plt.savefig(ipath+"Train&Test Dataset Images/"+str(ck)+"-Trajectory.jpg",bbox_inches='tight', dpi=100)
            ## These images are not submitted as these are used for annottation. Final image dataset is submitted directly
#             plt.show()
            print('Execution time for month',m, "Key", ck, ":", time.time() - st, 'seconds', ct)
        except:
            failed.append(ck)
            print('failed',m, "Key", ck, ":", time.time() - st, 'seconds', ct)

## 4. Generating training images

After Careful manual annotations of more than 1000 images from the plots generated above, The training image dataset is generated using the following sections

In [ ]:
## Importing annotations and coberting time stamps
annotations = pd.read_csv(ipath+"Train&Test Dataset Annotations.csv")
annotations["ST"] = annotations.apply(lambda x: time(x.Start_Hour,x.Start_Minute,0,0),axis=1)
annotations["ET"] = annotations.apply(lambda x: time(x.End_Hour, x.End_Minute,0,0),axis=1)
annotations.head(2)

In [ ]:
## Running for loop on annotation file to generate training images and saving in corresponding directories
failed = []
for i in range(0, len(annotations)):
    ck = annotations.at[i,"Crash_Key"]
    cr = annotations.at[i,"Crash"].upper()
    st = annotations.at[i,"ST"]
    et = annotations.at[i,"ET"]
    sm = annotations.at[i,"Start_Measure"]
    em = annotations.at[i,"End_Measure"]
    try:
        df = pd.read_csv(ipath+"Data/"+str(ck)+"-wejo.csv")
        df['capturedtimestamp'] = pd.to_datetime(df['capturedtimestamp'])
        df["time"] = df['capturedtimestamp'].apply(lambda x: x.time() )
        df1 = df[((df["time"]>= st) & (df["time"]<= et)) & ((df["measure"]>= sm) & (df["measure"]<= em))] 
        df1=df1[[ 'journeyid', 'capturedtimestamp', 'measure']]
        df1 =df1.sort_values(by = ["journeyid",'capturedtimestamp'])
        
        jids= df1[[ 'journeyid', 'measure']].groupby('journeyid').std().reset_index()
        jds= np.unique(jids[jids["measure"]>0.02]["journeyid"])
        df2 = df1[df1["journeyid"].isin(jds)==True]
        
        jids1= df2[[ 'journeyid', 'measure']].groupby('journeyid').count().reset_index()
        jds1 = np.unique(jids1[jids1["measure"] > 4]["journeyid"])
        df3 = df2[df2["journeyid"].isin(jds1)==True]
        
        df3.set_index('capturedtimestamp', inplace=True)
        fig, ax = plt.subplots(figsize=(50, 30))
        df3.groupby('journeyid')['measure'].plot(linewidth=4,color = 'black')
        plt.savefig(ipath+"Train&Test Dataset Images/"+str(cr)+"/"+str(ck)+"-"+str(i)+".jpg",bbox_inches='tight', dpi=100)
#         plt.show()
        print("Completed: ", i, ck , end = '\r')
    except:
        failed.append(i)
        print("failed: ",i, ck)

## 5. Convolutiona Nueral Network

We used Convolution Neural Networks(CNN) for image classification. In order to go further for model building we need to have a tensorflow environment. The following are the steps need to be followed to enable tensorflow environment.

In [ ]:
# You need to have numpy version which is less that 1.20.0 
#to run the below mentioned tensor flow model image data generator.
!pip install numpy==1.19.5 

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os
import numpy as np

This section of code(below) is used for constructing a Convolutional Neural Network (CNN) using Keras. 
Each line of the code adds a different layer to the CNN, designed for image recognition tasks.

In [ ]:
# Initialize a Sequential model
# This will serve as the foundation for our neural network,
# where layers will be added in a linear fashion.
classifier=Sequential()

#Layer 1
classifier.add(Convolution2D(32,(3,3),input_shape=(224,224,3), activation='relu')) #line adds the first convolution layer with 32 filters of size 3x3. The input_shape=(224,224,3) specifies that our input will be 224x224 RGB images (3 channels).
classifier.add(MaxPooling2D(pool_size=(2,2)))# max pooling layer with a pool size of 2x2. 

#layer 2
classifier.add(Convolution2D(32,(3,3), activation='relu'))#line adds the first convolution layer with 32 filters of size 3x3. The input_shape=(224,224,3) specifies that our input will be 224x224 RGB images (3 channels).
classifier.add(MaxPooling2D(pool_size=(2,2)))# max pooling layer with a pool size of 2x2.

#Layer 3
classifier.add(Convolution2D(64,(3,3), activation='relu'))# increases the number of filters to 64
classifier.add(MaxPooling2D(pool_size=(2,2)))# max pooling layer with a pool size of 2x2.

In [ ]:
# Flattening the 2D feature maps into a 1D vector for the dense layers
classifier.add(Flatten())

In [ ]:
# Adding a dense layer with 32 units and ReLU activation for further processing
classifier.add(Dense(units=32,activation='relu'))

# Adding another dense layer with 64 units and ReLU activation for complexity
classifier.add(Dense(units=64,activation='relu'))

# Increasing to 128 units in the dense layer for more complex feature learning
classifier.add(Dense(units=128,activation='relu'))

# Expanding to 256 units in the dense layer to enhance model's learning capacity
classifier.add(Dense(units=256,activation='relu'))

# Final dense layer with 2 units and sigmoid activation for binary classification
classifier.add(Dense(units=2,activation='sigmoid'))

# Compiling the model with the Adam optimizer, categorical cross-entropy loss, and tracking accuracy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Importing ImageDataGenerator for data augmentation and preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setting up the image data generator for training data with augmentation parameters
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.15,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

# Setting up the image data generator for test data with only rescaling
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
#Define data paths
train_path_dir=r"C:\Users\*****\Train&Test Dataset Images\Train"
test_path_dir=r"C:\Users\*****\Train&Test Dataset Images\Test"

In [ ]:
#Load datasets for model training.
training_set=train_datagen.flow_from_directory(train_path_dir,
                                               target_size=(224,224),
                                               batch_size=20,
                                               class_mode='categorical')

test_set=test_datagen.flow_from_directory(test_path_dir,target_size=(224,224),batch_size=10,class_mode='categorical')

In [ ]:
#model training set for 20 epochs and each each epoch is have 30 steps.
x=classifier.fit_generator(training_set,
                        steps_per_epoch=30,
                        epochs=20,
                        validation_data=test_set,
                        validation_steps=20)

In [ ]:
#Saving the model
classifier.save('Crash_non_crash.h5')

## 6 Testing the Model

In [ ]:
#Load Model
model = load_model('Crash_non_crash.h5')

In [ ]:
# Function to preprocess and predict an image
def predict_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values

    # Make a prediction
    prediction = model.predict(img_array)

    # Translate prediction to class label
    if prediction[0][0] > prediction[0][1]:
        return "crash"
    else:
        return "non-crash"

In [ ]:
# Mention the test dataset path for identification of either Crash or Non-Crash
path=r"C:\Users\*****\Train&Test Dataset Images\Test\Crash"
images=os.listdir(path)
for pic in images:
    image_path_address=os.path.join(path,pic)
    image_path = image_path_address
    result = predict_image(image_path)
    print(f"The image {pic} is classified as {result} image")

## 7. Evaluation

### 7.1 Generating Five-min trahectory images from 20 crashes other than training crashes

In [ ]:
## Calling the evaluation annotations for 20 crashes selected
annotations = pd.read_csv(ipath+"Evaluation Images.csv")
annotations["ST"] = annotations.apply(lambda x: time(int(x.Start_Hour),int(x.Start_Minute),0),axis=1)
annotations["ET"] = annotations.apply(lambda x: time(int(x.End_Hour), int(x.End_Minute),0,0),axis=1)
annotations.head(2)

In [ ]:
## Running for loop on evaluation annotation file to generate evaluation images at one minute interval for five_minute time periods and saving in corresponding directories

failed = []
for i in range(0, len(annotations)):
    ck = annotations.at[i,"Crash_Key"]
    st = annotations.at[i,"ST"]
    et = annotations.at[i,"ET"]
    t2 = (dt.datetime.combine(dt.date(1,1,1), st) + timedelta(minutes=10)).time()
    sm = annotations.at[i,"Start_Measure"]
    em = annotations.at[i,"End_Measure"]
    d = annotations.at[i,"Crash_time"]

    try:
        df = pd.read_csv(ipath+"Evaluation Images"+str(ck)+"-wejo.csv")
        df['capturedtimestamp'] = pd.to_datetime(df['capturedtimestamp'])
        df["time"] = df['capturedtimestamp'].apply(lambda x: x.time())
        df1 = df[((df["time"]>= st) & (df["time"]<= et)) & ((df["measure"]>= sm) & (df["measure"]<= em))] 
        df1=df1[[ 'journeyid', 'time', 'measure']]
        df1 =df1.sort_values(by = ["journeyid",'time'])

        jids= df1[[ 'journeyid', 'measure']].groupby('journeyid').std().reset_index()
        jds= np.unique(jids[jids["measure"]>0.01]["journeyid"])
        df2 = df1[df1["journeyid"].isin(jds)==True]

        jids1= df2[[ 'journeyid', 'measure']].groupby('journeyid').count().reset_index()
        jds1 = np.unique(jids1[jids1["measure"] > 1]["journeyid"])
        df3 = df2[df2["journeyid"].isin(jds1)==True]

        c=1

        while t2<=et:
            dfat=df3[(df3["time"]>=st) & (df3["time"]<=t2)]
            dfat.set_index('time', inplace=True)
            fig, ax = plt.subplots(figsize=(50, 30))
            dfat.groupby('journeyid')['measure'].plot(linewidth=4, color = 'black')
            plt.savefig(ipath+"Evaluation Images/"+str(ck)+"-"+str(c)+"-"+str(d)+"-"+str(t2.hour)+"-"+str(t2.minute)+".jpg",bbox_inches='tight', dpi=100)

            st = (dt.datetime.combine(dt.date(1,1,1), st) + timedelta(minutes=1)).time()
            t2 = (dt.datetime.combine(dt.date(1,1,1), t2) + timedelta(minutes=1)).time()
            c+=1
            print("Completed: ", ck ,i, t2, et, end = '\r')
    except:
        failed.append(i)
        print("failed: ",i, ck)

In [ ]:
## Defining function for crash prediction

def predict_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.0 

    # Make a prediction
    prediction = model.predict(img_array)

    # Translate prediction to class label
    if prediction[0][0] > prediction[0][1]:
        return "crash"
    else:
        return "non-crash"

In [ ]:


import glob

evaluation =pd.DataFrame(columns= ["Crash_Key","Sequence","Measure","Hour","Minute", "Prediction"])

files = glob.glob(ipath + "Evaluation Images/*")
for filename in files:
    fn=filename.split("\\")[-1].split('.')[0]
    pred = predict_image(filename)
#     print(pred)
    evaluation.loc[len(evaluation),:] = [fn.split("-")[0], fn.split("-")[1], fn.split("-")[2], fn.split("-")[3], fn.split("-")[4], pred]